In [4]:
import numpy as np
import math
import pymc3 as pm
import scipy.stats as stats
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1)

#Implementing BAST-RNN from https://arxiv.org/pdf/1711.00636.pdf
#Priors are all given on page 40
#Full conditionals are given on pages 45-46

#Questions
#1) Dimension reduction on pages 12-13?
#2) What should the embedding lag and length be for the input vector on page 9? 
#3) 

#Implementation
#1) Define priors and dimensions (page 40)
#2) Run PX-MCMC from Alg 1 (page 44) from paper sampling 100,000 iterations with the first 25,000 as burn-in, 
#thinning the samples so every fifth sample post burn-in is kept (keep 15,000). Monitor convergence with trace plots of 
#parameters and posterior forecasts. (Details on pages 40-46)
#Note that in PX-MCMC: 
#Step 1 is bullet point 1 in Appendix C (sampled with Metropolis-Hastings steps)
#Steps 2-4 are bullet point 2 in Appendix C (sampled with Metropolis-Hastings steps)
#Step 5 represents bullets 3-8 in Appendix C (samples with Gibbs steps)


#HIDDEN h_t = f(delta/abs(lambda_w)*W_(h_(t-1)) + U*X~_t) from equation (4) on page 8
print('HIDDEN')

#DIMENSIONS AND DETAILS
#f is activation function, use tanh np.tanh()
#h_t: n_h x n_h hidden layer
#n_h = 20 (given number of hidden units in paper)
#w_i,l: i = 1,...,n_h ; l = 1,...,n_h | square n_h x n_h matrix 
#u_i,r: i = 1,...,n_h ; r = 1,...,(m+1)*n_x + 1
#delta scaling parameter with Unif(0,1) prior
#lambda_w is largest eigenvalue of matrix W
#h_0 = 0 by definition
#X~_t from equation (5) on page 9 is a vector of [X_t, X_(t-tau), ..., X_(t-m*tau)]
#alpha is expansion parameter with dimensions i,l both of length n_h

#PRIORS
a_w = 0.2 #truncation of normal
a_u = 0.2
with pm.Model() as model:
    delta = pm.Uniform('delta', lower=0, upper=1)
    
    gammaw_il = pm.Bernoulli('gammaw_il', p=0.2)
    tnw1 = pm.TruncatedNormal('tnw1', mu=0, sd=1000, lower=-a_w, upper=a_w)
    tnw2 = pm.TruncatedNormal('tnw2', mu=0, sd=math.sqrt(.001), lower=-a_w, upper=a_w) #.001 or sqrt(.001)?
    w = gammaw_il*tnw1 + (1-gammaw_il)*tnw2 # should be matrix i x l 
    
    gammau_ir = pm.Bernoulli('gammau_ir', p=.025)
    tnu1 = pm.TruncatedNormal('tnu1', mu=0, sd=1000, lower=-a_u, upper=a_u)
    tnu2 = pm.TruncatedNormal('tnu2', mu=0, sd=math.sqrt(.0005), lower=-a_u, upper=a_u) #.0005 or sqrt(.0005)?
    u = gammau_ir*tnu1 + (1-gammaw_il)*tnu2 #should be matrix i x r

    
#DATA Y_t = mu + V_1*h_t + V_2*h_t^2 + epsilon_t from equation (3) on page 8
print('\n\nDATA')

#DIMENSIONS AND DETAILS
#mu ~ Gau(0, (sigma_mu)^2 * I 
#: n_y | 
#V_1: n_y x n_h
#V_2: n_y x n_h
#epsilon_t ~ Gau(0,R_t)
#R_t := (sigma_epsilon)^2 * I 
#(sigma_epsilon)^2 ~ IG(alpha_epsilon, beta_epsilon) 
#the above is inverse gamma with priors alpha_epsilon = .001 and beta_epsilon = .001

#output_size = 100

#PRIORS
with pm.Model() as model:
    sigma_e2 = pm.InverseGamma('sigma_e2', alpha=.001, beta=.001)
    mu = pm.Normal('mu', sd = 10) #doesn't need the identity matrix since output is just vector?
    epsilon_t = = pm.Normal('epsilon_t', sd=sigma_e2) #doesn't need the identity matrix since output is just a vector?
    
    gammav1_ki = pm.Bernoulli('gammav1_ki', p=0.5)
    v1ki1 = pm.Normal('v1ki1', mu=0, sd=math.sqrt(10)) #10 or sqrt(10)?
    v1ki1 = pm.Normal('tnw2', mu=0, sd=math.sqrt(.01)) #.01 or sqrt(.01)?
    v1 = gammav1_ki*v1ki1 + (1-gammaw_il)*v1ki1 # should be matrix y x h
    
    gammav2_ki = pm.Bernoulli('gammav2_ki', p=.25)
    v2ki1 = pm.Normal('v2ki1', mu=0, sd=math.sqrt(0.5), lower=-a_u, upper=a_u) #.5 or sqrt(.5)?
    v2ki2 = pm.Normal('v2ki2', mu=0, sd=math.sqrt(.05), lower=-a_u, upper=a_u) #.05 or sqrt(.05)?
    v2 = gammau_ir*tnu1 + (1-gammaw_il)*tnu2 #should be matrix y x h
    

#FULL CONDITIONALS BAST-RNN

#W, Gamma_W
with pm.model() as model:
    gammaw_il = pm.Bernoulli('gammaw_il', p=0.2)
    w_il = np.prod(math.exp(-(y_t-mu-v1*h_t+v2*(h_t)**2)*(y_t-mu-v1*h_t+v2*(h_t)**2)/(2*sigma_e2**2)))


SyntaxError: invalid syntax (<ipython-input-4-8b37332056c9>, line 79)

In [ ]:
#SAMPLING SETUP
with model:
    step = pm.Metropolis()
    trace = pm.sample(100000, step=step)
    burned_trace = trace[250000:] #burn first 250000
    thinned_trace = burned_trace[::5] #keep every 5th

In [8]:
help(pm.Normal)

Help on class Normal in module pymc3.distributions.continuous:

class Normal(pymc3.distributions.distribution.Continuous)
 |  Normal(name, *args, **kwargs)
 |  
 |  Univariate normal log-likelihood.
 |  
 |  The pdf of this distribution is
 |  
 |  .. math::
 |  
 |     f(x \mid \mu, \tau) =
 |         \sqrt{\frac{\tau}{2\pi}}
 |         \exp\left\{ -\frac{\tau}{2} (x-\mu)^2 \right\}
 |  
 |  Normal distribution can be parameterized either in terms of precision
 |  or standard deviation. The link between the two parametrizations is
 |  given by
 |  
 |  .. math::
 |  
 |     \tau = \dfrac{1}{\sigma^2}
 |  
 |  .. plot::
 |  
 |      import matplotlib.pyplot as plt
 |      import numpy as np
 |      import scipy.stats as st
 |      plt.style.use('seaborn-darkgrid')
 |      x = np.linspace(-5, 5, 1000)
 |      mus = [0., 0., 0., -2.]
 |      sds = [0.4, 1., 2., 0.4]
 |      for mu, sd in zip(mus, sds):
 |          pdf = st.norm.pdf(x, mu, sd)
 |          plt.plot(x, pdf, label=r'$\mu$ = 

In [6]:
import numpy as np
x= np.array([1, 2, 3])
y = np.array([5, 6, 7])

In [8]:
np.prod(x+y)

480